Import dependencies

In [1]:
import numpy as np
import os
import pandas as pd

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

Directory

In [2]:
os.chdir(os.getcwd()+'/images')
print(os.getcwd())

C:\Users\billk\Desktop\Git\spirochete_vision\images


In [3]:
pd.read_csv('data.csv', header=None)

,0,1,2,3,4,5,6,7,8,9,10
0,traingroup,filepath,object_class,xmin,ymin,TRX,TRY,xmax,ymax,BLX,BLY
1,TRAINING,1.jpg,1,0.7052083333333333,0.3885416666666667,NaN,NaN,0.7927083333333333,0.41770833333333335,NaN,NaN
2,VALIDATION,10.jpg,1,0.5208333333333334,0.4875,NaN,NaN,0.5760416666666667,0.559375,NaN,NaN
3,TRAINING,100.jpg,1,0.6096822995461422,0.6308623298033282,NaN,NaN,0.7246596066565809,0.7170953101361573,NaN,NaN
4,TRAINING,100.jpg,1,0.6671709531013615,0.7549167927382754,NaN,NaN,0.8290468986384266,0.8517397881996974,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
468,TRAINING,a7.jpg,1,0.43506493506493504,0.7792207792207793,NaN,NaN,0.551948051948052,0.9545454545454546,NaN,NaN
469,TRAINING,a7.jpg,1,0.3051948051948052,0.3961038961038961,NaN,NaN,0.43506493506493504,0.5714285714285714,NaN,NaN
470,TRAINING,a7.jpg,1,0.09740259740259741,0.01948051948051948,NaN,NaN,0.24025974025974026,0.24675324675324675,NaN,NaN
471,TRAINING,a8.jpg,1,0.1891891891891892,0.4648648648648649,NaN,NaN,0.5459459459459459,0.6594594594594595,NaN,NaN


Specify model

In [4]:
spec = model_spec.get('efficientdet_lite2')

Use data loader

In [5]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('data.csv')

Train model

In [6]:
model = object_detector.create(train_data, model_spec=spec, batch_size=9, epochs=100, train_whole_model=True, validation_data=validation_data)

Epoch 1/100
14/14 [==============================] - 64s 969ms/step - det_loss: 1.8154 - cls_loss: 1.1531 - box_loss: 0.0132 - reg_l2_loss: 0.0761 - loss: 1.8915 - learning_rate: 0.0096 - gradient_norm: 1.8961 - val_det_loss: 1.7086 - val_cls_loss: 1.1207 - val_box_loss: 0.0118 - val_reg_l2_loss: 0.0761 - val_loss: 1.7847
Epoch 2/100
14/14 [==============================] - 9s 653ms/step - det_loss: 1.6886 - cls_loss: 1.0590 - box_loss: 0.0126 - reg_l2_loss: 0.0761 - loss: 1.7646 - learning_rate: 0.0112 - gradient_norm: 2.6166 - val_det_loss: 1.5137 - val_cls_loss: 0.9454 - val_box_loss: 0.0114 - val_reg_l2_loss: 0.0761 - val_loss: 1.5898
Epoch 3/100
14/14 [==============================] - 9s 639ms/step - det_loss: 1.3152 - cls_loss: 0.7739 - box_loss: 0.0108 - reg_l2_loss: 0.0761 - loss: 1.3913 - learning_rate: 0.0112 - gradient_norm: 3.8900 - val_det_loss: 1.6226 - val_cls_loss: 0.9773 - val_box_loss: 0.0129 - val_reg_l2_loss: 0.0761 - val_loss: 1.6987
Epoch 4/100
14/14 [===========

Evaluate

In [7]:
model.evaluate(test_data)

1/1 [==============================] - 5s 5s/step



{'AP': 0.4638559,
 'AP50': 0.65441686,
 'AP75': 0.5835142,
 'APs': 0.3883964,
 'APm': 0.4887391,
 'APl': -1.0,
 'ARmax1': 0.33275864,
 'ARmax10': 0.5086207,
 'ARmax100': 0.67413795,
 'ARs': 0.56666666,
 'ARm': 0.6847826,
 'ARl': -1.0,
 'AP_/object_class': -1.0,
 'AP_/1': 0.51996756,
 'AP_/2': 0.4077442}

Export model

In [8]:
model.export(export_dir='.')

Evaluate exported model

In [9]:
model.evaluate_tflite('model.tflite', test_data)

15/15 [==============================] - 308s 21s/step



{'AP': 0.4571912,
 'AP50': 0.662716,
 'AP75': 0.5808992,
 'APs': 0.36610562,
 'APm': 0.48317173,
 'APl': -1.0,
 'ARmax1': 0.33275864,
 'ARmax10': 0.48793104,
 'ARmax100': 0.49655172,
 'ARs': 0.48333332,
 'ARm': 0.51086956,
 'ARl': -1.0,
 'AP_/object_class': -1.0,
 'AP_/1': 0.510422,
 'AP_/2': 0.4039604}